In [ ]:
!pip install --upgrade datasketch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import wordpunct_tokenize
from collections import defaultdict
from nltk import tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
import random
import pylab
from sklearn.metrics import log_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm_notebook
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

In [ ]:
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTest.csv")
corpus_test.shape
corpus_test[5370:5374]

,Id,Content
5370,5370,What role does music play in our life?\n
5371,5371,Which is the best coaching for MP psc?\n
5372,5372,Which mail server is used for messaging in AT&...
5373,5373,What are the programming languages dev uses on...


In [ ]:
corpus_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTrain.csv")
corpus_train.shape
#corpus_train.head()

(531990, 2)

In [ ]:
corpus_test['Content'] = corpus_test['Content'].str.strip()
corpus_test['Content'] = corpus_test['Content'].str.lower()

corpus_train['Content'] = corpus_train['Content'].str.strip()
corpus_train['Content'] = corpus_train['Content'].str.lower()

In [ ]:
spec_chars = ["!", '"', "#", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<",
              "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~", "–"]

for char in spec_chars:
    corpus_test['Content'] = corpus_test['Content'].str.replace(char, '')

for char in spec_chars:
    corpus_train['Content'] = corpus_train['Content'].str.replace(char, '')

In [ ]:
comb_ques = corpus_train['Content'].append(corpus_test['Content']).reset_index()
comb_ques.head()

,index,Content
0,0,how many people are going towards using phones...
1,1,what audio format should i use for getting aud...
2,2,what is the corporate culture like at edwards ...
3,3,what is the best barbecue in kansas city
4,4,can i combine the output of two bolts to one b...


In [ ]:
set_dict={} # maps question id (eg 'm23') to set representation of question
set_dict_test={}
count=1
for question in tqdm_notebook([x for x in comb_ques['Content'] if type(x)==str]):
    temp_list = []
    for shingle in question.split(' '):
        if shingle not in stop_words:
            temp_list.append(shingle.lower())
    if count < 531990:
      set_dict["m{0}".format(count)] = set(temp_list)
    else:
      set_dict_test["m{0}".format(count)] = set(temp_list)
    count +=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
# Create minHash signatures

num_perm = 32
min_dict = {} # maps question id (eg 'm23') to min hash signatures
count2 = 1
for val in tqdm_notebook(set_dict.values()): 
    m = MinHash(num_perm=num_perm)
    for shingle in val:
        m.update(shingle.encode('utf8'))
    min_dict["m{}".format(count2)] = m
    count2+=1

min_dict_test = {}
for val in tqdm_notebook(set_dict_test.values()): 
    m = MinHash(num_perm=num_perm)
    for shingle in val:
        m.update(shingle.encode('utf8'))
    min_dict_test["m{}".format(count2)] = m
    count2+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
# Create LSH index
#from train corpus

lsh = MinHashLSH(threshold=0.8, num_perm=num_perm)
for key in tqdm_notebook(min_dict.keys()):
    lsh.insert(key,min_dict[key])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
for key in tqdm_notebook(min_dict_test.keys()):
  lsh.insert(key,min_dict_test[key])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [ ]:
len(min_dict)

531989

In [ ]:
def create_cand_pairs():
    big_list = []
    #query the test corpus
    for query in min_dict_test.keys():
        bucket = lsh.query(min_dict_test[query])
        #if len(bucket)==1:
            #big_list.append([bucket[0],"None"])
        if len(bucket)>1:
            first_val = bucket[0]
            for val in bucket[1:]:
                second_val = val
                big_list.append([first_val,second_val])
    return big_list
        
        
cand_pairs = create_cand_pairs()

In [ ]:
import re
count = 0
for c in cand_pairs: 
  if int(re.sub("[^0-9]", "", c[0])) > 531989 and int(re.sub("[^0-9]", "", c[1])) < 531989 :
    count = count + 1 
    #print(c)
    continue
  elif int(re.sub("[^0-9]", "", c[1])) > 531989 and int(re.sub("[^0-9]", "", c[0])) > 531989:
    count = count + 1 
    #print(c)
print("candidate duplicate pairs = " + str(count))

candidate duplicate pairs = 530


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_test['Content'])
vectorized_test_content = vectorizer.transform(corpus_test['Content'])
print(vectorized_test_content.shape)

(5374, 9011)


In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_train['Content'])
vectorized_train_content = vectorizer.transform(corpus_train['Content'])
print(vectorized_train_content.shape)

(531990, 10000)


In [ ]:
corpus_compined = pd.DataFrame(corpus_test['Content'].append(corpus_train['Content'], ignore_index = True)) 
#corpus_compined['Content']

In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_compined['Content'])
vectorized_combined_content = vectorizer.transform(corpus_compined['Content'])
print(vectorized_combined_content.shape)

(537364, 10000)


Exact Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
count = 0
i = 0
similar = []
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[5374:100000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8: 
      similar.append(i)
      count = count + 1
      break
  i = i + 1
print(count)

1098


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[100000:200000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

1914


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[200000:300000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

2176


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[300000:400000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

2381


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[400000:])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

2604


In [ ]:
print("Duplicates identified with exact cosine: " + str(len(similar)))

Duplicates identified: 2196


Exact Jaccard Similarity

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score, accuracy_score

In [ ]:
vectorized_test_arr = vectorized_combined_content[0:5374].toarray()

In [ ]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
for j in range(5374):
  c = jaccard_score(vectorized_test_arr[5373], vectorized_test_arr[j], average='macro')
  if c > 0.8: 
    count = count + 1

In [ ]:
count = 0
temp = []
for i in range(5374, 6000):
  temp = vectorized_combined_content[i].toarray()
  for j in range(5374):
    c = jaccard_score(temp[0], vectorized_test_arr[j], average='macro')
    if c > 0.8: 
      count = count + 1
      break
print(count)

38


MinHash LSH

In [ ]:
!pip install --upgrade datasketch

In [ ]:
from datasketch import MinHash, MinHashLSH

In [ ]:
num_permutations = 32
count = 0
removed_items_indexes = []
for i in range(5374, 537364):
  set1 = set(corpus_train['Content'][i].split(" "))
  m1 = MinHash(num_perm=num_permutations)

  for d in set1:
      m1.update(d.encode('utf8'))

  for j in range(5374):
    set2 = set(corpus_test['Content'][j].split(" "))
    m2 = MinHash(num_perm=num_permutations)

    for d in set2:
        m2.update(d.encode('utf8'))

    # Create LSH index
    lsh = MinHashLSH(threshold=0.5, num_perm=num_permutations)
    lsh.insert("m2", m2)
    result = lsh.query(m1)
    if len(result) > 0: 
      count = count + 1
    else: 
      removed_ items_indexes.append(j)
print(count)

LSH-Cosine

In [ ]:
from sklearn.neighbors import LSHForest

In [ ]:
train_indexes_arr = []
for j in range(5374, 537364):
  temp = []
  for i in range(len(vectorized_combined_content[j].nonzero()[0])):
    temp.append(vectorized_combined_content[j].nonzero()[1][i])
  train_indexes_arr.append(temp)

In [ ]:
test_indexes_arr = []
for j in range(5374):
  temp = []
  for i in range(len(vectorized_combined_content[j].nonzero()[0])):
    temp.append(vectorized_combined_content[j].nonzero()[1][i])
  test_indexes_arr.append(temp)

In [ ]:
lshf = LSHForest()
lshf.fit(train_indexes_arr) 
distances, indices = lshf.kneighbors(test_indexes_arr, n_neighbors=2)

LSH-Jaccard

In [ ]:
#updated arrays, contain only the test rows with lsh similarity > 0.5
count = 0
for i in range(5374, 537364):
  for j in range(5374):
    c = jaccard_score(vectorized_train_arr[i], vectorized_test_arr[j])
    if c > 0.8: 
      count = count + 1
print(count)